In [1]:
import pandas as pd
# main_df_path = "../../../../RMI-Data/20250604_dfjson_score.xlsx"
rootcause_df_path = "../../../../RMI-Data/20250604_dfjson_rootcause.xlsx"
process_df_path = "../../../../RMI-Data/20250604_dfjson_process.xlsx"

# main_df = pd.read_excel(main_df_path)
ori_rootcause_df = pd.read_excel(rootcause_df_path)
ori_process_df = pd.read_excel(process_df_path)
# rename ori_rootcause_df description to root_cause_description 
ori_rootcause_df.rename(columns={'description': 'root_cause_description'}, inplace=True)
 
# merge main_df and rootcause_df and process_df

In [2]:
ori_process_df.head()
ori_rootcause_df.head()

,Unnamed: 0,control_name_en,company_id,company_name,country,business_sector_name,revenue,ebitda,assessment_id,assessment_form_id,...,risk_level,risk_level_L,group_name,risk_mapping,riskcat_mapping,subcat_mapping,ref_id,root_cause_name,root_cause_description,rootcause_index
0,0,NaN,413,AERA1,Thailand,Services,722.91,-269.63,1324,2001,...,Moderate risk,2,AERA1,Accounting errors,operational_risk,Operation,NaN,Insufficient training of accounting staff on a...,NaN,437
1,1,NaN,413,AERA1,Thailand,Services,722.91,-269.63,1324,2001,...,Moderate risk,2,AERA1,Accounting errors,operational_risk,Operation,NaN,"Weak internal controls, including lack of segr...","Human error from manual work, no system to hel...",437
2,2,NaN,413,AERA1,Thailand,Services,722.91,-269.63,1322,1995,...,Moderate risk,2,AERA1,Air pollution,strategic_risk,Environment,NaN,NaN,การใช้งานสารเคมีที่มีองค์ประกอบของไอระเหย,395
3,3,NaN,413,AERA1,Thailand,Services,722.91,-269.63,1323,2000,...,High risk,3,AERA1,Business interruption from fire hazards,operational_risk,Operation,NaN,NaN,Fire from external parties such as Ramkamhaeng...,432
4,4,NaN,413,AERA1,Thailand,Services,722.91,-269.63,1323,2000,...,High risk,3,AERA1,Business interruption from fire hazards,operational_risk,Operation,NaN,Outdated Fire Detection and Alarm Systems,NaN,432


In [3]:
# process_df is groupby ['group_name', 'username', 'assessment_form_id', 'question_name', 'risk_mapping'] from ori_process_df
rootcause_df = ori_rootcause_df.groupby(['group_name', 'username', 'assessment_form_id', 'question_name', 'risk_mapping']).agg({'root_cause_name': set,"root_cause_description":set}).reset_index()
rootcause_df.head()

# rootcause_df is groupby ['group_name', 'username', 'assessment_form_id', 'question_name', 'risk_mapping'] from ori_rootcause_df
process_df = ori_process_df.groupby(['group_name', 'username', 'assessment_form_id', 'question_name', 'risk_mapping']).agg({'process_name': set,"process_description":set}).reset_index()
process_df.head()
print(f"{process_df.shape=}")
print(f"{rootcause_df.shape=}")

process_df.shape=(4101, 7)
rootcause_df.shape=(4175, 7)


In [4]:
import os
import pandas as pd

# df_path = "../../../../RMI-Data/20250604_RMI_result_Q1_2025_maxriskbybu.xlsx"
df_path = "../../../../RMI-Data/20250604_RMI_result_Q1_2025_alldata.xlsx"
ori_df = pd.read_excel(df_path)
# ori_df["company"] = "PCG"
# ori_df["process_desc"] = ""
# ori_df["rootcause_desc"] = ""
# select_country = "China"
select_country = "Thailand"

# ori_df = ori_df[ori_df["country"] == select_country]

#decoded_string = ascii_string.encode('latin1').decode('unicode_escape')
# apply this to risk_mapping and answer_description
# fill nan ori_df["answer_description"] with ""
ori_df["answer_description"] = ori_df["answer_description"].fillna("")
ori_df["risk_mapping"] = ori_df["risk_mapping"].apply(lambda x: x.encode('utf-8').decode('unicode_escape'))
# replace risk_mapping 'Lack of customerâ\x80\x99s credit screening' with Lack of customer’s credit screening
ori_df["risk_mapping"] = ori_df["risk_mapping"].apply(lambda x: x.replace('Lack of customerâ\x80\x99s credit screening', 'Lack of customer’s credit screening'))

# ori_df drop process_answer and cause_answer
ori_df = ori_df.drop(columns=['process_answer', 'cause_answer'])


# merge ori_df and process_df and rootcause_df
ori_df = pd.merge(ori_df, process_df, on=['group_name', 'username', 'assessment_form_id', 'question_name', 'risk_mapping'], how='left')
ori_df = pd.merge(ori_df, rootcause_df, on=['group_name', 'username', 'assessment_form_id', 'question_name', 'risk_mapping'], how='left')
# fill nan with empty {} for process_name root_cause_name root_cause_description process_description



# rename process_name to process_answer and root_cause_name to rootcause_answer
ori_df.rename(columns={'process_name': 'process_answer', 'root_cause_name': 'cause_answer'}, inplace=True)

# rename root_cause_description to rootcause_desc and process_description to process_desc
ori_df.rename(columns={'root_cause_description': 'rootcause_desc', 'process_description': 'process_desc'}, inplace=True)
ori_df["cause_answer"] = ori_df["cause_answer"].apply(lambda x: set() if pd.isna(x) else x)
ori_df["rootcause_desc"] = ori_df["rootcause_desc"].apply(lambda x: set() if pd.isna(x) else x)
ori_df["process_answer"] = ori_df["process_answer"].apply(lambda x: set() if pd.isna(x) else x)
ori_df["process_desc"] = ori_df["process_desc"].apply(lambda x: set() if pd.isna(x) else x)
df = ori_df.copy()
print(df.shape)
df.head()

(5268, 72)


,Unnamed: 0,control_name_en,existing_control_name,existing_control_description,existing_control_process_answer,existing_control_cause_answer,existing_control_owner,existing_control_control_design,existing_control_effective_operating,existing_control_ref_id,...,risk_level,risk_level_L,group_name,risk_mapping,riskcat_mapping,subcat_mapping,process_answer,process_desc,cause_answer,rootcause_desc
0,437,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Moderate risk,2,AERA1,Accounting errors,operational_risk,Operation,"{Finance and Accounting, Legal, Corporate Comm...",{nan},"{Weak internal controls, including lack of seg...","{Human error from manual work, no system to he..."
1,395,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Moderate risk,2,AERA1,Air pollution,strategic_risk,Environment,"{Health, Safety, and Environment (HSE), Mainte...",{nan},{nan},{การใช้งานสารเคมีที่มีองค์ประกอบของไอระเหย}
2,432,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,High risk,3,AERA1,Business interruption from fire hazards,operational_risk,Operation,"{Legal, Regulatory to contact SRT, Service, Ma...","{Operation staff to train the fire drill, To e...",{Negligence or Human Error in Handling Flammab...,{There are SRT areas that have flammable mater...
3,455,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Low risk,1,AERA1,Business interruption from fire hazards,operational_risk,Operation,"{Finance and Accounting, Legal, Business devel...",{nan},{Negligence or Human Error in Handling Flammab...,{nan}
4,456,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Moderate risk,2,AERA1,Business interruption from flood,operational_risk,Operation,"{Business development, Marketing and Sales, He...",{nan},{Inadequate Regular Maintenance of Infrastruct...,{nan}


In [5]:
# raise

In [6]:
ori_df['cause_answer'][0]

{'Insufficient training of accounting staff on accounting principles, standards, and procedures',
 'Weak internal controls, including lack of segregation of duties, inadequate review processes, or reliance on manual systems prone to errors'}

In [7]:
import json
# convert process_answer to dict
debug_df = ori_df.copy()
# loop each row to convert process_answer to dict to see which one fails
for index, row in debug_df.iterrows():
    try:
        json.loads(row["process_answer"])
    except:
        print(index)
        print(row["process_answer"])
        break

# convert process_answer to dict

0
{'Finance and Accounting', 'Legal', 'Corporate Communication', 'Business development', 'Facilities Management', 'Engineering', 'Information Technology (IT)', 'Health, Safety, and Environment (HSE)', 'Maintenance', 'Human Resources', 'Procurement'}


In [8]:
# fail_dict_string = """{"2": {"ref_id": 2, "name": "Finance and Accounting", "description": "ประเมินสถานะทางการเงินเบื้องต้นของลูกค้า (แม้จะเป็นบริษัทในเครือ)
# ออกใบแจ้งหนี้และติดตามการรับชำระค่าไฟฟ้ารายเดือน
# บริหารจัดการลูกหนี้คงค้าง (Account Receivable Management)
# วิเคราะห์และรายงานผลกระทบต่อกระแสเงินสดและงบการเงิน กรณีลูกหนี้ชำระล่าช้าหรือผิดนัด
# อาจต้องตั้งค่าเผื่อหนี้สงสัยจะสูญ (Allowance for Doubtful Accounts) หากความเสี่ยงสูงขึ้น"}, "4": {"ref_id": 4, "name": "Legal", "description": "ร่างและตรวจสอบสัญญา PPA ให้ครอบคลุมเงื่อนไขการชำระเงิน, ดอกเบี้ยผิดนัด, และแนวทางการดำเนินการกรณีผิดนัดชำระหนี้
# ให้คำปรึกษาด้านกฎหมายเกี่ยวกับสิทธิและหน้าที่ตามสัญญา
# ดำเนินการทางกฎหมายเพื่อติดตามหนี้สินในกรณีจำเป็น"}, "11": {"ref_id": 11, "name": "Business Development", "description": "ประเมินความน่าเชื่อถือและความมั่นคงทางการเงินของบริษัทลูกค้าในเครือฯ ที่จะเข้ามาทำสัญญา PPA ในระยะยาว
# มีส่วนร่วมในการกำหนดโครงสร้างสัญญาและเงื่อนไขทางการค้าที่อาจส่งผลต่อความเสี่ยงด้านเครดิต
# รวบรวมข้อมูลเบื้องต้นเกี่ยวกับศักยภาพและสถานะของลูกค้า"}, "13": {"ref_id": 13, "name": "Planning", "description": ""}}"""
# print(fail_dict_string[130:140])
# fail_dict = json.loads(fail_dict_string)

In [9]:
# raise

In [10]:
df.columns.tolist()

['Unnamed: 0',
 'control_name_en',
 'existing_control_name',
 'existing_control_description',
 'existing_control_process_answer',
 'existing_control_cause_answer',
 'existing_control_owner',
 'existing_control_control_design',
 'existing_control_effective_operating',
 'existing_control_ref_id',
 'existing_control_external_organization_answer',
 'company_id',
 'company_name',
 'country',
 'business_sector_name',
 'revenue',
 'ebitda',
 'assessment_id',
 'assessment_form_id',
 'question_id',
 'assessment_status',
 'username',
 'email',
 'title',
 'question_name',
 'question_description',
 'is_custom',
 'sub_category_id',
 'category',
 'sub_category_name',
 'answer_description',
 'kri',
 'rating_score_answer',
 'mitigation_answer',
 'have_mitigation',
 'mitigation_name',
 'mitigation_description',
 'mitigation_cost_support',
 'mitigation_other_support',
 'mitigation_remark',
 'mitigation_start_date',
 'mitigation_end_date',
 'mitigation_attachment',
 'mitigation_owner',
 'mitigation_cause

In [11]:
# df group_name ="AERA1" and value_count risk
df = ori_df.copy()
# df = df[df["group_name"] == "AERA1"]
# df["risk_mapping"].value_counts()
df['country'].value_counts()
# df[df['group_name']=="PCG"]['answer_description']
# df['group_name'].value_counts().keys().tolist()


country
China       2686
Thailand    2582
Name: count, dtype: int64

In [12]:
# sentence = df[df['risk_mapping'] == "Accounting errors"]['cause_answer'].to_list()[2]
# sentence
# # apply this to sentence
# sentence = sentence.encode('latin1').decode('unicode_escape')
# sentence

In [13]:
df[df['risk_mapping'] == "Accounting errors"]['process_answer'].to_list()

[{'Business development',
  'Corporate Communication',
  'Engineering',
  'Facilities Management',
  'Finance and Accounting',
  'Health, Safety, and Environment (HSE)',
  'Human Resources',
  'Information Technology (IT)',
  'Legal',
  'Maintenance',
  'Procurement'},
 {'Finance and Accounting', 'Human Resources', 'Information Technology (IT)'},
 {'Finance and Accounting'},
 {'Finance and Accounting'},
 {'Human Resources'},
 {'Finance and Accounting', 'Production'},
 {'Finance and Accounting'},
 {'Finance and Accounting', 'Production'},
 {'Information Technology (IT)'},
 {'Engineering', 'Finance and Accounting'},
 {'Finance and Accounting'},
 {'Finance and Accounting', 'Information Technology (IT)'},
 {'Finance and Accounting'},
 {'Finance and Accounting', 'Human Resources', 'Legal'}]

In [14]:
ascii_string = "\\u0e2d\\u0e32\\u0e08\\u0e40\\u0e01\\u0e34\\u0e14\\u0e01\\u0e32\\u0e23\\u0e1c\\u0e34\\u0e14\\u0e1e\\u0e25\\u0e32\\u0e14\\u0e08\\u0e32\\u0e01\\u0e01\\u0e32\\u0e23\\u0e04\\u0e33\\u0e19\\u0e27\\u0e13 \\u0e41\\u0e15\\u0e48\\u0e40\\u0e19\\u0e37\\u0e48\\u0e2d\\u0e07\\u0e08\\u0e32\\u0e01\\u0e15\\u0e49\\u0e2d\\u0e07\\u0e1c\\u0e48\\u0e32\\u0e19\\u0e01\\u0e32\\u0e23\\u0e15\\u0e23\\u0e27\\u0e08\\u0e2a\\u0e2d\\u0e1a\\u0e2b\\u0e25\\u0e32\\u0e22\\u0e02\\u0e31\\u0e49\\u0e19\\u0e15\\u0e2d\\u0e19 \\u0e08\\u0e36\\u0e07\\u0e2a\\u0e32\\u0e21\\u0e32\\u0e23\\u0e16\\u0e15\\u0e23\\u0e27\\u0e08\\u0e2a\\u0e2d\\u0e1a\\u0e1e\\u0e1a\\u0e41\\u0e25\\u0e30\\u0e2a\\u0e32\\u0e21\\u0e32\\u0e23\\u0e16\\u0e41\\u0e01\\u0e49\\u0e44\\u0e02\\u0e44\\u0e14\\u0e49"
ascii_string="root_cause_2_OP041_mistakes_in_calculations,_data_entry,_or_judgment_errors_during_the_accounting_process"
ascii_string="root_cause_2_OP041_mistakes_in_calculations,_data_entry,_or_judgment_errors_during_the_accounting_process\\u0e2d\\u0e32\\u0e08\\u0e40\\u0e01\\u0e34\\u0e14\\u0e01\\u0e32\\u0e23\\u0e1c\\u0e34\\u0e14\\u0e1e\\u0e25\\u0e32\\u0e14\\u0e08\\u0e32\\u0e01\\u0e01\\u0e32\\u0e23\\u0e04\\u0e33\\u0e19\\u0e27\\u0e13 \\u0e41\\u0e15\\u0e48\\u0e40\\u0e19\\u0e37\\u0e48\\u0e2d\\u0e07\\u0e08\\u0e32\\u0e01\\u0e15\\u0e49\\u0e2d\\u0e07\\u0e1c\\u0e48\\u0e32\\u0e19\\u0e01\\u0e32\\u0e23\\u0e15\\u0e23\\u0e27\\u0e08\\u0e2a\\u0e2d\\u0e1a\\u0e2b\\u0e25\\u0e32\\u0e22\\u0e02\\u0e31\\u0e49\\u0e19\\u0e15\\u0e2d\\u0e19 \\u0e08\\u0e36\\u0e07\\u0e2a\\u0e32\\u0e21\\u0e32\\u0e23\\u0e16\\u0e15\\u0e23\\u0e27\\u0e08\\u0e2a\\u0e2d\\u0e1a\\u0e1e\\u0e1a\\u0e41\\u0e25\\u0e30\\u0e2a\\u0e32\\u0e21\\u0e32\\u0e23\\u0e16\\u0e41\\u0e01\\u0e49\\u0e44\\u0e02\\u0e44\\u0e14\\u0e49"

ascii_string = "จัดหาพนักงานที่มีความรู้ ความสารถ ที่เหมาะสมกับตำแหน่งงาน \\u0e2d\\u0e32\\u0e08\\u0e40\\u0e01\\u0e34\\u0e14\\u0e01\\u0e32\\u0e23\\u0e1c\\u0e34\\u0e14\\u0e1e\\u0e25\\u0e32\\u0e14\\u0e08\\u0e32\\u0e01\\u0e01\\u0e32\\u0e23\\u0e04\\u0e33\\u0e19\\u0e27\\u0e13 \\u0e41\\u0e15\\u0e48\\u0e40\\u0e19\\u0e37\\u0e48\\u0e2d\\u0e07\\u0e08\\u0e32\\u0e01\\u0e15\\u0e49\\u0e2d\\u0e07\\u0e1c\\u0e48\\u0e32\\u0e19\\u0e01\\u0e32\\u0e23\\u0e15\\u0e23\\u0e27\\u0e08\\u0e2a\\u0e2d\\u0e1a\\u0e2b\\u0e25\\u0e32\\u0e22\\u0e02\\u0e31\\u0e49\\u0e19\\u0e15\\u0e2d\\u0e19 \\u0e08\\u0e36\\u0e07\\u0e2a\\u0e32\\u0e21\\u0e32\\u0e23\\u0e16\\u0e15\\u0e23\\u0e27\\u0e08\\u0e2a\\u0e2d\\u0e1a\\u0e1e\\u0e1a\\u0e41\\u0e25\\u0e30\\u0e2a\\u0e32\\u0e21\\u0e32\\u0e23\\u0e16\\u0e41\\u0e01\\u0e49\\u0e44\\u0e02\\u0e44\\u0e14\\u0e49"
# The 'unicode_escape' codec is used to decode strings that contain
# Python-style Unicode escape sequences (e.g., \uXXXX).

decoded_string = ascii_string.encode('utf-8').decode('unicode_escape')

print(decoded_string)

à¸à¸±à¸à¸«à¸²à¸à¸à¸±à¸à¸à¸²à¸à¸à¸µà¹à¸¡à¸µà¸à¸§à¸²à¸¡à¸£à¸¹à¹ à¸à¸§à¸²à¸¡à¸ªà¸²à¸£à¸ à¸à¸µà¹à¹à¸«à¸¡à¸²à¸°à¸ªà¸¡à¸à¸±à¸à¸à¸³à¹à¸«à¸à¹à¸à¸à¸²à¸ อาจเกิดการผิดพลาดจากการคำนวณ แต่เนื่องจากต้องผ่านการตรวจสอบหลายขั้นตอน จึงสามารถตรวจสอบพบและสามารถแก้ไขได้


In [15]:
# debug_list = df.risk_mapping.to_list()
# save to txt
# df.answer_description.to_list()
# df.process_answer.to_list()
df.cause_answer.to_list()
# with open("debug_list.txt", "w") as f:
#     for item in debug_list:
#         f.write(item + "\n")

[{'Insufficient training of accounting staff on accounting principles, standards, and procedures',
  'Weak internal controls, including lack of segregation of duties, inadequate review processes, or reliance on manual systems prone to errors'},
 {nan},
 {'Negligence or Human Error in Handling Flammable Materials',
  'Outdated Fire Detection and Alarm Systems',
  nan},
 {'Negligence or Human Error in Handling Flammable Materials'},
 {'Inadequate Regular Maintenance of Infrastructure',
  'Insufficient Flood Monitoring and Alert Systems',
  'Lack of Awareness of Flood Risks'},
 {nan},
 {nan},
 {'Inadequate Awareness of Natural Disaster Risks'},
 {'Absence of Remote Work Policies and Procedures',
  'Lack of Flexible Work Arrangements'},
 {nan},
 {nan},
 {'Human Error',
  'Inadequate Compliance Management Systems',
  'Inadequate Physical Control',
  'Insufficient Knowledge Management',
  'Lack of Effective Monitoring or Enforcement Tools',
  'Lack of Encryption',
  'Lack of IT Audit',
  'La

In [16]:
df.riskcat_mapping.value_counts()

riskcat_mapping
operational_risk    3791
strategic_risk      1097
market_risk          193
liquidity_risk       103
credit_risk           84
Name: count, dtype: int64

In [17]:
selected_risk_cat = [
    "Operational Risk",
    "Strategic Risk",
    "Credit Risk",
    "Market Risk",
    "Liquidity Risk",
]
replace_risk_cat = {
    "operational_risk": "Operational Risk",
    "strategic_risk": "Strategic Risk",
    "credit_risk": "Credit Risk",
    "market_risk": "Market Risk",
    "liquidity_risk": "Liquidity Risk",
}



In [18]:
file_name = os.path.basename(df_path)
date_stamp = file_name.split("_")[0]
date_stamp

'20250604'

In [19]:
# calc risk score
# Select columns that start with 'financial' but do not end with '_note'
# riskscore_columns = df.loc[:, (df.columns.str.startswith('Financial') | df.columns.str.startswith('NonFinancial') | df.columns.str.startswith('Likelihood')) & ~df.columns.str.endswith('_notes') & ~df.columns.str.endswith('_justification')]
df = ori_df.copy()
df.riskcat_mapping.replace(replace_risk_cat, inplace=True)
df.riskcat_mapping.value_counts()
# convert columns name to lower case
# df.columns = df.columns.str.lower()

# rename columns
df.rename(columns={"risk_level": "risk_level_temp", "risk_level_L": "risk_level_L_temp",}, inplace=True)
df.rename(columns={ "risk_level_temp": "risk_level_label","risk_level_L_temp": "risk_level"}, inplace=True)




/tmp/ipykernel_28294/3209166167.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df.riskcat_mapping.replace(replace_risk_cat, inplace=True)


In [20]:
df["risk_level"].value_counts()

risk_level
1    2848
2    1972
3     418
4      30
Name: count, dtype: int64

In [21]:
df.rename(
    columns={
        "company":"company_bu",
        "group_name":"company_real",
        "riskcat_mapping": "risk_cat",
        "risk_mapping": "risk",
        "answer_description": "risk_desc",
        "cause_answer": "rootcause",
        "process_answer": "process",
        "score_impact":"impact_combined",
        "score_likelihood":"likelihood_combined",
        "country":"company",
        # "Control": "control_name",
        # "Control Description": "control_des",
        # "EC-Root Cause": "control_rootcause",
        # "EC-Process": "control_process",
    },
    inplace=True,
)

In [22]:
debug_df = df[df["company_real"] == "PCG"]
debug_df = debug_df[debug_df["risk"].str.lower() == "poor supply quality"]
debug_df[["risk", "risk_score", "likelihood_combined", "impact_combined"]]

,risk,risk_score,likelihood_combined,impact_combined
1732,Poor supply quality,3,1,3
1837,Poor supply quality,6,3,2
1838,Poor supply quality,6,3,2
1842,Poor supply quality,8,4,2
1852,Poor supply quality,3,3,1


In [23]:
# raise

In [24]:
# df2 = df.groupby(["company", "risk"], as_index=False).agg(
#     {
#         "risk_desc": set,
#         "risk_cat": "first",
#         # "iscustom": set,  #'first', # risk
#         # "company_sector": set,
#         # "risk_subcat": set,
#         # "control_name": set,
#         # "control_des": set,
#         "rootcause": set,
#         "rootcause_desc": set,
#         "process": set,
#         "process_desc": set,
#         "risk_level": max,
#         "likelihood_combined": max,
#         "impact_combined": max,
#         # "impact_fin_combined": max,
#         # "impact_nonfin_combined": max,
#     }
# )
# # TODO: check if other df# are recalculating this since max(impact)*max(likelihood) != max(risk_score)
# df2["risk_score"] = df2["impact_combined"] * df2["likelihood_combined"]
# # df2["risk_score_fin"] = df2["impact_fin_combined"] * df2["likelihood_combined"]
# # df2["risk_score_nonfin"] = df2["impact_nonfin_combined"] * df2["likelihood_combined"]

# df2["risk_level"] = df2.apply(determine_risk_level, type="combine", axis=1)
# # df2["risk_level_fin"] = df2.apply(determine_risk_level, type="fin", axis=1)
# # df2["risk_level_nonfin"] = df2.apply(determine_risk_level, type="nonfin", axis=1)

In [25]:
df[["risk_desc", "risk_cat", "impact_combined", "risk_score", "risk_level", "rootcause", "rootcause_desc", "process", "process_desc"]].isna().sum()


risk_desc          0
risk_cat           0
impact_combined    0
risk_score         0
risk_level         0
rootcause          0
rootcause_desc     0
process            0
process_desc       0
dtype: int64

In [26]:

# Define the function to determine the risk level
def determine_risk_level(row, type):
    if type == "fin":
        riskscore = "risk_score_fin"
        riskimpact = "impact_fin_combined"
    elif type == "nonfin":
        riskscore = "risk_score_nonfin"
        riskimpact = "impact_nonfin_combined"
    else:
        riskscore = "risk_score"
        riskimpact = "impact_combined"
    if row[riskscore] >= 20:
        return 4
    elif 10 <= row[riskscore] <= 16:
        return 3
    elif 4 <= row[riskscore] <= 9:
        return 2
    elif (0 < row[riskscore] < 4) or (row[riskscore] == 4 and row[riskimpact] == 2):
        return 1
    elif row[riskscore] == 0:
        return 0
    # else:
    #     return 'Undefined'
# score_method = "MAX"
score_method = "RMI"
if score_method == "MAX":
    df2 = df.groupby(["company", "risk"], as_index=False).agg(
        {
            "risk_desc": set,
            "risk_cat": "first",
            # "iscustom": set,  #'first', # risk
            # "company_sector": set,
            # "risk_subcat": set,
            # "control_name": set,
            # "control_des": set,
            "rootcause": set,
            "rootcause_desc": set,
            "process": set,
            "process_desc": set,
            "risk_level": max,
            "likelihood_combined": max,
            "impact_combined": max,
            # "impact_fin_combined": max,
            # "impact_nonfin_combined": max,
        }
    )
    # TODO: check if other df# are recalculating this since max(impact)*max(likelihood) != max(risk_score)
    df2["risk_score"] = df2["impact_combined"] * df2["likelihood_combined"]
    # df2["risk_score_fin"] = df2["impact_fin_combined"] * df2["likelihood_combined"]
    # df2["risk_score_nonfin"] = df2["impact_nonfin_combined"] * df2["likelihood_combined"]

    df2["risk_level"] = df2.apply(determine_risk_level, type="combine", axis=1)
    # df2["risk_level_fin"] = df2.apply(determine_risk_level, type="fin", axis=1)
    # df2["risk_level_nonfin"] = df2.apply(determine_risk_level, type="nonfin", axis=1)
elif score_method == "RMI":
    df2 = df.copy()
    df2["risk_score"] = df2["impact_combined"] * df2["likelihood_combined"]
    df2_sorted = df2.sort_values(
        by=["company_real", "risk", "risk_score", "impact_combined"],
        ascending=[True, True, False, False],
    )
    # KeyError: "['risk_cat', 'rootcause', 'process_desc'] not in index"
    
    df2 = df2_sorted.groupby(["company_real", "risk","company"], as_index=False).agg(
        {
            "risk_desc": set,
            # 'iscustom': set, #'first', # risk
            # "company_sector": set,
            "risk_cat": set,
            "impact_combined": "first",
            # 'impact_fin_combined': 'max',
            # 'impact_nonfin_combined': 'max',
            # 'likelihood_combined': 'max',
            # 'impact_combined': 'mean',
            # 'likelihood_combined': 'mean',
            "risk_score": "max",
            # 'risk_score_fin' : 'max',
            # 'risk_score_nonfin' : 'max',
            "risk_level": "max",
            # 'risk_level_fin' : 'max',
            # 'risk_level_nonfin' : 'max',
            # "control_name": set,
            # "control_des": set,
            # "control_rootcause": set,
            # "control_process": set,
            # 'KRI_name' : set,
            # 'KRI_desc' : set,
            "rootcause": lambda x: set().union(*x),
            "rootcause_desc": lambda x: set().union(*x),
            "process": lambda x: set().union(*x),
            "process_desc": lambda x: set().union(*x),
            # 'est_revenue': 'first'  # Retain the revenue for the company
        }
    )

    # TODO: check if other df# are recalculating this since max(impact)*max(likelihood) != max(risk_score)

    # Normally, this method won't work: if L*I = 3*3 = 9 and 5*2 = 10 -> max score = 10, likelihood = 10/3 = 3.33
    # but it works here because we sorted the risk_score and impact
    df2["likelihood_combined"] = df2["risk_score"] / df2["impact_combined"]


else:
    raise ValueError(f"Invalid score method: {score_method}")
df2["risk_level"] = df2.apply(determine_risk_level, type="combine", axis=1)

In [27]:
print(df2.shape)
df2.head()

(3115, 13)


,company_real,risk,company,risk_desc,risk_cat,impact_combined,risk_score,risk_level,rootcause,rootcause_desc,process,process_desc,likelihood_combined
0,AERA1,Accounting errors,Thailand,{A significant risk is that multiple users may...,{Operational Risk},2,6,2,"{Weak internal controls, including lack of seg...","{Human error from manual work, no system to he...","{Finance and Accounting, Legal, Corporate Comm...",{nan},3.0
1,AERA1,Air pollution,Thailand,{การใช้งานสารที่มีไอระเหย},{Strategic Risk},2,6,2,{nan},{การใช้งานสารเคมีที่มีองค์ประกอบของไอระเหย},"{Health, Safety, and Environment (HSE), Mainte...",{nan},3.0
2,AERA1,Business interruption from fire hazards,Thailand,"{จากสภาพแวดล้อมและอุปกรณ์ที่เริ่มเสื่อมสภาพ, T...",{Operational Risk},4,12,3,{Negligence or Human Error in Handling Flammab...,{There are SRT areas that have flammable mater...,"{Finance and Accounting, Legal, Business devel...","{Operation staff to train the fire drill, Fire...",3.0
3,AERA1,Business interruption from flood,Thailand,{Although AERA1 station is elevated except for...,{Operational Risk},4,8,2,{Inadequate Regular Maintenance of Infrastruct...,{nan},"{Maintenance, Marketing and Sales, Health, Saf...",{nan},2.0
4,AERA1,Business interruption from labor dispute,Thailand,{Possibility to happen is low but the impact i...,{Operational Risk},4,8,2,{nan},{Rail industry is labor intensive and industri...,{Human Resources},{nan},2.0


In [28]:
df2.company.value_counts()

company
China       1835
Thailand    1280
Name: count, dtype: int64

In [29]:
df2_5 = df2.copy()
df2_5["risk_score"] = df2_5["impact_combined"] * df2_5["likelihood_combined"]
df2_5_sorted = df2_5.sort_values(
    by=["company_real", "risk", "risk_score", "impact_combined"],
    ascending=[True, True, False, False],
)
# KeyError: "['risk_cat', 'rootcause', 'process_desc'] not in index"

df2_5 = df2_5_sorted.groupby([ "risk","company"], as_index=False).agg(
    {
        "risk_desc": lambda x: set().union(*x),
        # 'iscustom': set, #'first', # risk
        # "company_sector": set,
        # count number of company
        "company_real":set, 
        "risk_cat": lambda x: set().union(*x),
        "impact_combined": "first",
        # 'impact_fin_combined': 'max',
        # 'impact_nonfin_combined': 'max',
        # 'likelihood_combined': 'max',
        # 'impact_combined': 'mean',
        # 'likelihood_combined': 'mean',
        "risk_score": "max",
        # 'risk_score_fin' : 'max',
        # 'risk_score_nonfin' : 'max',
        "risk_level": "max",
        # 'risk_level_fin' : 'max',
        # 'risk_level_nonfin' : 'max',
        # "control_name": set,
        # "control_des": set,
        # "control_rootcause": set,
        # "control_process": set,
        # 'KRI_name' : set,
        # 'KRI_desc' : set,
        "rootcause": lambda x: set().union(*x),
        "rootcause_desc": lambda x: set().union(*x),
        "process": lambda x: set().union(*x),
        "process_desc": lambda x: set().union(*x),
        # 'est_revenue': 'first'  # Retain the revenue for the company
    }
)

# TODO: check if other df# are recalculating this since max(impact)*max(likelihood) != max(risk_score)

# Normally, this method won't work: if L*I = 3*3 = 9 and 5*2 = 10 -> max score = 10, likelihood = 10/3 = 3.33
# but it works here because we sorted the risk_score and impact
df2_5["likelihood_combined"] = df2_5["risk_score"] / df2_5["impact_combined"]

df2_5["risk_level"] = df2_5.apply(determine_risk_level, type="combine", axis=1)

In [30]:
# df3 = df2[
    # df2["company"].isin(
    #     [
    #         "PCG",
    #     ]
    # )
# ]
df3 = df2_5.copy()
print(df3.shape)
df3.company.value_counts()

(401, 13)


company
Thailand    204
China       197
Name: count, dtype: int64

In [31]:
df3.head()

,risk,company,risk_desc,company_real,risk_cat,impact_combined,risk_score,risk_level,rootcause,rootcause_desc,process,process_desc,likelihood_combined
0,Accounting errors,China,"{财务系统中，对财务交易的记录、计算或报告的不准确, 财务系统漏洞（如ERP数据录入错误）...","{产城服务与开发 Development Services, 四川区 Sichuan Are...",{Operational Risk},1,2.0,1,"{Mistakes in calculations, data entry, or judg...","{内控薄弱, 流程繁琐容易出错, nan}","{Information Technology (IT), Finance and Acco...","{实施财务共享中心与自动化对账系统, 一是关键岗位人员变动风险，如高管频繁离职，会影响公司经...",2.0
1,Accounting errors,Thailand,"{ไม่บันทึกต้นทุน ไม่ทันเวลา , ขาดเรื่องระบบ หร...","{PCG, Altervim, W&W, AERA1, TDL, CPLI, ECI, KP...",{Operational Risk},2,12.0,3,{Issues with accounting software or systems th...,{ไม่มี ละเลยระบบความปลอดภัยด้านไซเบอร์และการป้...,"{Production, Finance and Accounting, Legal, Co...",{ไม่มี ละเลยระบบความปลอดภัยด้านไซเบอร์และการป้...,6.0
2,Adverse outcomes of AI technologies,China,{目前，人工智能技术还不够成熟，应用场景受到一定的局限，有时会增加流程，有时也可能会带来误操...,"{植物一条龙 Crop Intergration Business, 商业地产 Commer...",{Strategic Risk},1,6.0,2,"{未准确使用AI, 人工智能与应用场景结合不紧密，智能度还不能达到预想的程度, Lack o...","{人工决策效率偏低, 未充分应用人工智能技术, nan}","{Production, Legal, Engineering, Research and ...","{人工决策效率偏低, 缺乏对人工职能应用的判断, nan}",6.0
3,Adverse outcomes of AI technologies,Thailand,{เนื่องจากพนักงานมีการใช้ AI และ Chat GPT ในกา...,"{W&W, TDL, Advance AQ, CPLI, Advance-AHBG}",{Strategic Risk},1,8.0,2,{Lack of established ethical standards for AI ...,{nan},"{Quality Assurance, Engineering, Facilities Ma...",{nan},8.0
4,After-sale service-related dissatisfaction,China,{销售产品后送至客户处，客户可能存在对包装破损，数量不足，后续技术服务指导存在不满意的情况，...,"{河南区 Henan Area, 正大生物 CP Bio, 种植事业 Plant Busin...",{Operational Risk},2,6.0,2,{Failure to follow up with customers post-purc...,"{客户对售后的结果不满意, 对客户的重视程度不够, nan}","{Production, Warehouse, Packing, Corporate Com...","{销售知情但未及时和公司汇报, 售后服务能力没有及时跟进。, 产品价格、产品品质、技术服务的...",3.0


In [32]:
df3.columns

Index(['risk', 'company', 'risk_desc', 'company_real', 'risk_cat',
       'impact_combined', 'risk_score', 'risk_level', 'rootcause',
       'rootcause_desc', 'process', 'process_desc', 'likelihood_combined'],
      dtype='object')

In [33]:
select_columns = [
    "company",
    "company_real",
    "risk_cat",
    "risk",
    "risk_desc",
    "rootcause",
    "rootcause_desc",
    "process",
    "process_desc",
    "risk_level",
]
df4 = df3[select_columns]
print(df4.shape)
df4.head()

(401, 10)


,company,company_real,risk_cat,risk,risk_desc,rootcause,rootcause_desc,process,process_desc,risk_level
0,China,"{产城服务与开发 Development Services, 四川区 Sichuan Are...",{Operational Risk},Accounting errors,"{财务系统中，对财务交易的记录、计算或报告的不准确, 财务系统漏洞（如ERP数据录入错误）...","{Mistakes in calculations, data entry, or judg...","{内控薄弱, 流程繁琐容易出错, nan}","{Information Technology (IT), Finance and Acco...","{实施财务共享中心与自动化对账系统, 一是关键岗位人员变动风险，如高管频繁离职，会影响公司经...",1
1,Thailand,"{PCG, Altervim, W&W, AERA1, TDL, CPLI, ECI, KP...",{Operational Risk},Accounting errors,"{ไม่บันทึกต้นทุน ไม่ทันเวลา , ขาดเรื่องระบบ หร...",{Issues with accounting software or systems th...,{ไม่มี ละเลยระบบความปลอดภัยด้านไซเบอร์และการป้...,"{Production, Finance and Accounting, Legal, Co...",{ไม่มี ละเลยระบบความปลอดภัยด้านไซเบอร์และการป้...,3
2,China,"{植物一条龙 Crop Intergration Business, 商业地产 Commer...",{Strategic Risk},Adverse outcomes of AI technologies,{目前，人工智能技术还不够成熟，应用场景受到一定的局限，有时会增加流程，有时也可能会带来误操...,"{未准确使用AI, 人工智能与应用场景结合不紧密，智能度还不能达到预想的程度, Lack o...","{人工决策效率偏低, 未充分应用人工智能技术, nan}","{Production, Legal, Engineering, Research and ...","{人工决策效率偏低, 缺乏对人工职能应用的判断, nan}",2
3,Thailand,"{W&W, TDL, Advance AQ, CPLI, Advance-AHBG}",{Strategic Risk},Adverse outcomes of AI technologies,{เนื่องจากพนักงานมีการใช้ AI และ Chat GPT ในกา...,{Lack of established ethical standards for AI ...,{nan},"{Quality Assurance, Engineering, Facilities Ma...",{nan},2
4,China,"{河南区 Henan Area, 正大生物 CP Bio, 种植事业 Plant Busin...",{Operational Risk},After-sale service-related dissatisfaction,{销售产品后送至客户处，客户可能存在对包装破损，数量不足，后续技术服务指导存在不满意的情况，...,{Failure to follow up with customers post-purc...,"{客户对售后的结果不满意, 对客户的重视程度不够, nan}","{Production, Warehouse, Packing, Corporate Com...","{销售知情但未及时和公司汇报, 售后服务能力没有及时跟进。, 产品价格、产品品质、技术服务的...",2


In [34]:
# select first row of column risk_desc
cell = df4.iloc[4, 2]
# cell = df4.iloc[2, 3]
# cell = df4.iloc[3, 3]
# print(type(cell))
# print(cell)
import json
import traceback
from typing import Union


def list_string_to_list(list_string) -> tuple[bool, Union[list, str]]:
    is_list = False
    if ((list_string.count("'") % 2 != 0) and (list_string.count("'") != 0)) or (
        (list_string.count('"') % 2 != 0) and (list_string.count('"') != 0)
    ):

        return is_list, list_string
    # print(f"pass list_string :{type(list_string)} {list_string}")
    is_list = True
    # swap "'" to '"' and swap '"' to "'"
    list_string = list_string.replace('"', "###")
    list_string = list_string.replace("'", '"')
    list_string = list_string.replace("###", "'")
    try:
        list_cell = json.loads(list_string)
    except:
        is_list = False
        # print(f"here list_string : {list_string}")
        # print(traceback.format_exc())
        return is_list, list_string
    return is_list, list_cell


def cell_to_list(cell):
    empty_element_list = ["", "-"]
    if type(cell) == set:
        new_cell = []
        for list_string_candidate in cell:
            if list_string_candidate in empty_element_list:
                continue
            try:
                # if list_string count "'" or '"' have to be even and not 0
                # print(f"list_string_candidate : >>>{list_string_candidate}<<<")

                is_list, new_list_string_candidate = list_string_to_list(
                    list_string_candidate
                )
                # print(new_list_string_candidate == list_string_candidate)
                # print("is_list", is_list)
                # print(f"new_list_string_candidate : >>>{new_list_string_candidate}<<<")
                if is_list is False:
                    new_cell.append(new_list_string_candidate)
                    continue
                for i in new_list_string_candidate:
                    if i in empty_element_list:
                        continue

                    is_list, i = list_string_to_list(i)
                    # print(f"i : {is_list}>>>{i}<<<")
                    if is_list is False:
                        new_cell.append(i)
                        continue
                    new_cell.extend(i)

            except:
                # print(f"list_string : {list_string}")
                # print(traceback.format_exc())
                new_cell.append(list_string)
        return new_cell
    return cell


result = cell_to_list(cell)
print(cell_to_list(cell))
print(type(cell_to_list(cell)))

# print(cell_to_list(cell)[0])
# print(type(cell_to_list(cell)[0]))

['Operational Risk']
<class 'list'>


In [35]:
result

['Operational Risk']

In [36]:
list_string = "ขั้นตอนในการเคลมสินค้า เพื่อขอชดเชย มีหลายขั้นตอน ซึ่งขึ้นอยู่แต่ละปัญหาของสินค้า"
if ((list_string.count("'") % 2 != 0) and (list_string.count("'") != 0)) or (
    (list_string.count('"') % 2 != 0) and (list_string.count('"') != 0)
):
    print("yes")

In [37]:
# df5 is df4 with pass each cell with cell_to_list
# convert set of list_string to list
df5 = df4.applymap(cell_to_list)
df5.head()

/tmp/ipykernel_28294/139688506.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df5 = df4.applymap(cell_to_list)


,company,company_real,risk_cat,risk,risk_desc,rootcause,rootcause_desc,process,process_desc,risk_level
0,China,"[产城服务与开发 Development Services, 四川区 Sichuan Are...",[Operational Risk],Accounting errors,"[财务系统中，对财务交易的记录、计算或报告的不准确, 财务系统漏洞（如ERP数据录入错误）...","[Mistakes in calculations, data entry, or judg...","[内控薄弱, 流程繁琐容易出错, ขั้นตอนในการเคลมสินค้า เพื่อข...","[Information Technology (IT), Finance and Acco...","[实施财务共享中心与自动化对账系统, 一是关键岗位人员变动风险，如高管频繁离职，会影响公司经...",1
1,Thailand,"[PCG, Altervim, W&W, AERA1, TDL, CPLI, ECI, KP...",[Operational Risk],Accounting errors,"[ไม่บันทึกต้นทุน ไม่ทันเวลา , ขาดเรื่องระบบ หร...",[Issues with accounting software or systems th...,[ไม่มี ละเลยระบบความปลอดภัยด้านไซเบอร์และการป้...,"[Production, Finance and Accounting, Legal, Co...",[ไม่มี ละเลยระบบความปลอดภัยด้านไซเบอร์และการป้...,3
2,China,"[植物一条龙 Crop Intergration Business, 商业地产 Commer...",[Strategic Risk],Adverse outcomes of AI technologies,[目前，人工智能技术还不够成熟，应用场景受到一定的局限，有时会增加流程，有时也可能会带来误操...,"[未准确使用AI, 人工智能与应用场景结合不紧密，智能度还不能达到预想的程度, Lack o...","[人工决策效率偏低, 未充分应用人工智能技术, ขั้นตอนในการเคลมสินค้า...","[Production, Legal, Engineering, Research and ...","[人工决策效率偏低, 缺乏对人工职能应用的判断, ขั้นตอนในการเคลมสินค้...",2
3,Thailand,"[W&W, TDL, Advance AQ, CPLI, Advance-AHBG]",[Strategic Risk],Adverse outcomes of AI technologies,[เนื่องจากพนักงานมีการใช้ AI และ Chat GPT ในกา...,[Lack of established ethical standards for AI ...,[ขั้นตอนในการเคลมสินค้า เพื่อขอชดเชย มีหลายขั้...,"[Quality Assurance, Engineering, Facilities Ma...",[ขั้นตอนในการเคลมสินค้า เพื่อขอชดเชย มีหลายขั้...,2
4,China,"[河南区 Henan Area, 正大生物 CP Bio, 种植事业 Plant Busin...",[Operational Risk],After-sale service-related dissatisfaction,[销售产品后送至客户处，客户可能存在对包装破损，数量不足，后续技术服务指导存在不满意的情况，...,[Failure to follow up with customers post-purc...,"[客户对售后的结果不满意, 对客户的重视程度不够, ขั้นตอนในการเคลมสินค...","[Production, Warehouse, Packing, Corporate Com...","[销售知情但未及时和公司汇报, 售后服务能力没有及时跟进。, 产品价格、产品品质、技术服务的...",2


In [38]:
# df6 convert list to string with join
df6 = df5.applymap(lambda x: ",".join(x) if isinstance(x, list) else x)
df6.head()

/tmp/ipykernel_28294/3479902738.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df6 = df5.applymap(lambda x: ",".join(x) if isinstance(x, list) else x)


,company,company_real,risk_cat,risk,risk_desc,rootcause,rootcause_desc,process,process_desc,risk_level
0,China,"产城服务与开发 Development Services,四川区 Sichuan Area,...",Operational Risk,Accounting errors,"财务系统中，对财务交易的记录、计算或报告的不准确, 财务系统漏洞（如ERP数据录入错误）,财...","Mistakes in calculations, data entry, or judgm...","内控薄弱,流程繁琐容易出错,ขั้นตอนในการเคลมสินค้า เพื่อขอชด...","Information Technology (IT),Finance and Accoun...","实施财务共享中心与自动化对账系统,一是关键岗位人员变动风险，如高管频繁离职，会影响公司经营方...",1
1,Thailand,"PCG,Altervim,W&W,AERA1,TDL,CPLI,ECI,KPI/KSP",Operational Risk,Accounting errors,"ไม่บันทึกต้นทุน ไม่ทันเวลา ,ขาดเรื่องระบบ หรือ...",Issues with accounting software or systems tha...,ไม่มี ละเลยระบบความปลอดภัยด้านไซเบอร์และการป้อ...,"Production,Finance and Accounting,Legal,Corpor...",ไม่มี ละเลยระบบความปลอดภัยด้านไซเบอร์และการป้อ...,3
2,China,"植物一条龙 Crop Intergration Business,商业地产 Commerci...",Strategic Risk,Adverse outcomes of AI technologies,目前，人工智能技术还不够成熟，应用场景受到一定的局限，有时会增加流程，有时也可能会带来误操作...,"未准确使用AI,人工智能与应用场景结合不紧密，智能度还不能达到预想的程度,Lack of e...","人工决策效率偏低,未充分应用人工智能技术,ขั้นตอนในการเคลมสินค้า เพ...","Production,Legal,Engineering,Research and Deve...","人工决策效率偏低,缺乏对人工职能应用的判断,ขั้นตอนในการเคลมสินค้า เ...",2
3,Thailand,"W&W,TDL,Advance AQ,CPLI,Advance-AHBG",Strategic Risk,Adverse outcomes of AI technologies,เนื่องจากพนักงานมีการใช้ AI และ Chat GPT ในการ...,Lack of established ethical standards for AI i...,ขั้นตอนในการเคลมสินค้า เพื่อขอชดเชย มีหลายขั้น...,"Quality Assurance,Engineering,Facilities Manag...",ขั้นตอนในการเคลมสินค้า เพื่อขอชดเชย มีหลายขั้น...,2
4,China,"河南区 Henan Area,正大生物 CP Bio,种植事业 Plant Business...",Operational Risk,After-sale service-related dissatisfaction,销售产品后送至客户处，客户可能存在对包装破损，数量不足，后续技术服务指导存在不满意的情况，或...,Failure to follow up with customers post-purch...,"客户对售后的结果不满意,对客户的重视程度不够,ขั้นตอนในการเคลมสินค้า ...","Production,Warehouse,Packing,Corporate Communi...","销售知情但未及时和公司汇报,售后服务能力没有及时跟进。,产品价格、产品品质、技术服务的售后,...",2


In [39]:
df7 = df6.copy()


# new_rootcause is 'rootcause : ' + rootcause column +\n+ 'rootcause_desc : ' rootcause_desc if it not '' empty string
# fundction to apply
def merge_rootcause(row):
    rootcause = row["rootcause"]
    rootcause_desc = row["rootcause_desc"]
    if rootcause == "" and rootcause_desc == "":
        return ""
    elif rootcause == "":
        return "rootcause_desc :" + rootcause_desc
    elif rootcause_desc == "":
        return "rootcause :" + rootcause
    else:
        return "rootcause :" + rootcause + "\n" + "rootcause_desc :" + rootcause_desc


df7["new_rootcause"] = df7.apply(merge_rootcause, axis=1)


def merge_process(row):
    process = row["process"]
    process_desc = row["process_desc"]
    if process == "" and process_desc == "":
        return ""
    elif process == "":
        return "process_desc :" + process_desc
    elif process_desc == "":
        return "process :" + process
    else:
        return "process :" + process + "\n" + "process_desc :" + process_desc


df7["new_process"] = df7.apply(merge_process, axis=1)
# remove rootcause and process column
df7.drop(
    ["rootcause", "rootcause_desc", "process", "process_desc"], axis=1, inplace=True
)
# rename new_rootcause and new_process to rootcause and process
df7.rename(
    columns={"new_rootcause": "rootcause", "new_process": "process"}, inplace=True
)
df7.head()

,company,company_real,risk_cat,risk,risk_desc,risk_level,rootcause,process
0,China,"产城服务与开发 Development Services,四川区 Sichuan Area,...",Operational Risk,Accounting errors,"财务系统中，对财务交易的记录、计算或报告的不准确, 财务系统漏洞（如ERP数据录入错误）,财...",1,"rootcause :Mistakes in calculations, data entr...","process :Information Technology (IT),Finance a..."
1,Thailand,"PCG,Altervim,W&W,AERA1,TDL,CPLI,ECI,KPI/KSP",Operational Risk,Accounting errors,"ไม่บันทึกต้นทุน ไม่ทันเวลา ,ขาดเรื่องระบบ หรือ...",3,rootcause :Issues with accounting software or ...,"process :Production,Finance and Accounting,Leg..."
2,China,"植物一条龙 Crop Intergration Business,商业地产 Commerci...",Strategic Risk,Adverse outcomes of AI technologies,目前，人工智能技术还不够成熟，应用场景受到一定的局限，有时会增加流程，有时也可能会带来误操作...,2,"rootcause :未准确使用AI,人工智能与应用场景结合不紧密，智能度还不能达到预想的程...","process :Production,Legal,Engineering,Research..."
3,Thailand,"W&W,TDL,Advance AQ,CPLI,Advance-AHBG",Strategic Risk,Adverse outcomes of AI technologies,เนื่องจากพนักงานมีการใช้ AI และ Chat GPT ในการ...,2,rootcause :Lack of established ethical standar...,"process :Quality Assurance,Engineering,Facilit..."
4,China,"河南区 Henan Area,正大生物 CP Bio,种植事业 Plant Business...",Operational Risk,After-sale service-related dissatisfaction,销售产品后送至客户处，客户可能存在对包装破损，数量不足，后续技术服务指导存在不满意的情况，或...,2,rootcause :Failure to follow up with customers...,"process :Production,Warehouse,Packing,Corporat..."


In [40]:
risk_df_path = "data/RMI-V2-Translate_20250508.xlsx"
risk_df = pd.read_excel(risk_df_path, sheet_name="Risks")
risk_cause_mapping_df = pd.read_excel(risk_df_path, sheet_name="Risk_Cause_mapping")

selected_cols = [
    "Risk-EN",
    "Description-EN",
]

# risk_df["Risk-category"]# replace Operational risk with Operational Risk
risk_df["Risk-category"] = risk_df["Risk-category"].replace(
    "Operational risk", "Operational Risk"
)
for idx, row in risk_df.iterrows():
    risk_en = row["Risk-EN"]
    # risk_cause_mapping_df column RiskName get all RiskCause
    risk_cause_mapping_df_row = risk_cause_mapping_df[
        risk_cause_mapping_df["RiskName"] == risk_en
    ]
    # get all RiskCause
    risk_cause_list = risk_cause_mapping_df_row["RiskCause"].tolist()
    risk_cause_str = "\n".join(["- " + i for i in risk_cause_list])
    # add risk_cause_str to column Root cause-EN
    risk_df.loc[idx, "Root cause-EN"] = risk_cause_str


print(risk_df.columns)
risk_and_risk_desc_df = risk_df[selected_cols]
# loop each row in risk_and_risk_desc_df:
# apply strip to all columns
risk_and_risk_desc_df = risk_and_risk_desc_df.applymap(lambda x: x.strip())

risk_and_risk_desc_df.head()
# create map_dict
risk_to_desc_dict = risk_and_risk_desc_df.set_index("Risk-EN")[
    "Description-EN"
].to_dict()
len(risk_to_desc_dict)

/home/ford/cro_rmi_improvement_feature/.venv/lib/python3.12/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/home/ford/cro_rmi_improvement_feature/.venv/lib/python3.12/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Index(['GroupingIndex', 'Grouping Question Name', 'Code', 'Risk-category',
       'Risk-Sub-Cat-EN', 'Risk-EN', 'Risk-TH', 'Risk-CN', 'Description-EN',
       'Description-TH', 'Description-CN', 'Unnamed: 11', 'Root cause-EN'],
      dtype='object')


/tmp/ipykernel_28294/3047557379.py:31: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  risk_and_risk_desc_df = risk_and_risk_desc_df.applymap(lambda x: x.strip())


314

In [41]:
df8 = df7.copy()
# print number cell == NaN
print(df8.isnull().sum().sum())
# risk column map with risk_to_desc_dict
# apply strip to df8["risk"]
df8["risk"] = df8["risk"].str.strip()
df8["risk_desc_catalog"] = df8["risk"].map(risk_to_desc_dict)

df8.head()
# is nan in risk_desc_catalog
print(df8["risk_desc_catalog"].isnull().sum())
# fill risk_desc_catalog nan with ""
df8["risk_desc_catalog"] = df8["risk_desc_catalog"].fillna("")
df8["risk_desc_catalog"] = df8["risk_desc_catalog"].str.strip()
# print row that risk_desc_catalog is nan
df8[df8["risk_desc_catalog"].isnull()]
# concat risk_desc_catalog and risk_desc
df8["risk_desc"] = df8["risk_desc_catalog"] + " " + df8["risk_desc"]
# strip df8["risk_desc"]
df8["risk_desc"] = df8["risk_desc"].str.strip()
# remove risk_desc_catalog
df8.drop("risk_desc_catalog", axis=1, inplace=True)
all_risk_cat = risk_df["Risk-category"].unique().tolist()
# check is all df8['risk_cat'] in all_risk_cat
assert all([x in all_risk_cat for x in df8["risk_cat"].unique().tolist()])
# add new data to df8 from risk_and_risk_cat_and_risk_desc_df
print(df8.shape)

new_rows = []
for index, row in risk_df.iterrows():
    risk_cat = row["Risk-category"]
    if risk_cat not in selected_risk_cat:
        continue
    risk_desc = row["Description-EN"]
    risk_name = row["Risk-EN"]
    risk_rootcause = row["Root cause-EN"]
    data = {
        # company	risk_cat	risk	risk_desc	rootcause	process
        "company": f"risk_catalog-{date_stamp}",
        "company_real": f"risk_catalog-{date_stamp}",
        "risk_cat": risk_cat,
        "risk": risk_name,
        "risk_desc": risk_desc,
        "rootcause": risk_rootcause,
        "process": "",
        "risk_level": 0,
    }
    new_rows.append(data)
df8 = pd.concat([df8, pd.DataFrame(new_rows)], ignore_index=True)
print(df8.shape)
print("nan", df8.isnull().sum().sum())

df8.head()

0
50
(401, 8)
(635, 8)
nan 0


,company,company_real,risk_cat,risk,risk_desc,risk_level,rootcause,process
0,China,"产城服务与开发 Development Services,四川区 Sichuan Area,...",Operational Risk,Accounting errors,Accounting error refers to inaccuracies in the...,1,"rootcause :Mistakes in calculations, data entr...","process :Information Technology (IT),Finance a..."
1,Thailand,"PCG,Altervim,W&W,AERA1,TDL,CPLI,ECI,KPI/KSP",Operational Risk,Accounting errors,Accounting error refers to inaccuracies in the...,3,rootcause :Issues with accounting software or ...,"process :Production,Finance and Accounting,Leg..."
2,China,"植物一条龙 Crop Intergration Business,商业地产 Commerci...",Strategic Risk,Adverse outcomes of AI technologies,Intended or unintended negative consequences o...,2,"rootcause :未准确使用AI,人工智能与应用场景结合不紧密，智能度还不能达到预想的程...","process :Production,Legal,Engineering,Research..."
3,Thailand,"W&W,TDL,Advance AQ,CPLI,Advance-AHBG",Strategic Risk,Adverse outcomes of AI technologies,Intended or unintended negative consequences o...,2,rootcause :Lack of established ethical standar...,"process :Quality Assurance,Engineering,Facilit..."
4,China,"河南区 Henan Area,正大生物 CP Bio,种植事业 Plant Business...",Operational Risk,After-sale service-related dissatisfaction,The risk of customers experiencing dissatisfac...,2,rootcause :Failure to follow up with customers...,"process :Production,Warehouse,Packing,Corporat..."


In [42]:
all([x in all_risk_cat for x in df8["risk_cat"].unique().tolist()])

True

In [43]:
all_risk_cat

['Operational Risk',
 'Strategic Risk',
 'Market Risk',
 'Credit Risk',
 'Liquidity Risk',
 'A. Setting up business\n\n',
 'B. Business operation',
 'C. Compliance and regulator',
 'D. Policy',
 'E. Raising Concerns and Seeking Guidance',
 'F. Review, Assessment and Improvement']

In [44]:
df8["risk_cat"].unique().tolist()

['Operational Risk',
 'Strategic Risk',
 'Liquidity Risk',
 'Credit Risk',
 'Market Risk']

In [45]:
df.shape

(5268, 72)

In [46]:
df8.shape

(635, 8)

In [47]:
df_final = df8.copy()

In [48]:
df_final.company.value_counts()

company
risk_catalog-20250604    234
Thailand                 204
China                    197
Name: count, dtype: int64

In [49]:
df_final[df_final['company']==f"risk_catalog-{date_stamp}"]

,company,company_real,risk_cat,risk,risk_desc,risk_level,rootcause,process
401,risk_catalog-20250604,risk_catalog-20250604,Operational Risk,Supply shortage,Shortages or defects in essential raw material...,0,- Supplier / Vendor shortage\n- Logistics and ...,
402,risk_catalog-20250604,risk_catalog-20250604,Operational Risk,Poor supply quality,The risk that the supplier will fail to delive...,0,- Selection of suppliers based solely on cost ...,
403,risk_catalog-20250604,risk_catalog-20250604,Operational Risk,Low bargaining power with suppliers,Low bargaining power with suppliers can lead t...,0,- Weak negotiation skills or strategy\n- Poor ...,
404,risk_catalog-20250604,risk_catalog-20250604,Operational Risk,Machinery / equipment failure and damage,"Machinery or equipment damage, malfunction, or...",0,- Improper operation\n- Errors in handling by ...,
405,risk_catalog-20250604,risk_catalog-20250604,Operational Risk,Machinery / equipment loss,The loss or theft of machinery or equipment us...,0,"- Mistakes such as mislabeling, misplacement\n...",
...,...,...,...,...,...,...,...,...
630,risk_catalog-20250604,risk_catalog-20250604,Operational Risk,Inadequate Management of OT Third-party,This risk refers to situation where third-part...,0,"- Third-party vulnerabilities (e.g., third-par...",
631,risk_catalog-20250604,risk_catalog-20250604,Operational Risk,Ineffective OT Operations Management,This risk refers to situation where the activi...,0,- Lack of standardized OT Policies/Processes/P...,
632,risk_catalog-20250604,risk_catalog-20250604,Operational Risk,OT Hardware Failure,This risk refers to the malfunction of OT-rela...,0,- Accidental damage during routine maintenance...,
633,risk_catalog-20250604,risk_catalog-20250604,Operational Risk,OT Software Failure,"This risk refers to OT software errors, bugs, ...",0,- Inadequate software testing or quality assur...,


In [50]:
# convert df6:dataframe to data_dict
result_list = df_final.to_dict(orient="records")
result_list[:3]

[{'company': 'China',
  'company_real': '产城服务与开发 Development Services,四川区 Sichuan Area,正大制药 CP Pharmacy,北京总部 Headquarters',
  'risk_cat': 'Operational Risk',
  'risk': 'Accounting errors',
  'risk_desc': "Accounting error refers to inaccuracies in the recording, calculation, or reporting of financial transactions within an organization's accounting system. These errors can vary from simple mathematical mistakes to more complex issues like misapplication of accounting principles or estimation errors. 财务系统中，对财务交易的记录、计算或报告的不准确, 财务系统漏洞（如ERP数据录入错误）,财务数据计算不准确,在税务方面，可能通过线上服务平台虚开发票，包括平台主动配合、受串通欺诈或缺乏业务真实性证明材料等情况；还可能虚开农产品收购发票，借此增加成本、套取资金用于行贿或偷逃税款，且虚开常与行贿相互牵连，面临刑事处罚。在财务数据方面，部分企业存在虚增营收、利润或成本的造假行为，如通过压货提前确认收入、不当核算贸易收入等手段虚增营收利润，或虚增成本以少交所得税、套取资金用于行贿等 。此外，还可能存在关联交易存疑、应收账款高企、研发投入异常等财务异常情况，这些财务错误风险不仅会使企业面临监管处罚、法律责任，还会影响企业信誉和投资者信心。",
  'risk_level': 1,
  'rootcause': 'rootcause :Mistakes in calculations, data entry, or judgment errors during the accounting process,Issues with accounting software or syste

In [51]:
# save it as json
import os

dir_path = os.getcwd()
save_file_path = f"{dir_path}/result/{date_stamp}-company_risk_data.json"
# save result_list as json
import json

with open(save_file_path, "w") as f:
    json.dump(result_list, f, ensure_ascii=False)